## Imports

In [2]:
import pandas as pd
import backtrader as bt
import requests
import json
import matplotlib

In [3]:
# from warnings import simplefilter
# # from pandas.core.common import SettingWithCopyWarning
# simplefilter(action="ignore", category=FutureWarning)
# # warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


## Data Fetching

In [4]:
url = "https://api.tokenmetrics.com/v1/trading-indicator"
querystring = {"tokens": "3375", "limit": "1830"}
header = {"api_key": "tm-5484e45f-ac87-471d-b3e3-748b27bea9ed"}
response = requests.request("GET", url, headers=header, params=querystring)


In [5]:
data = json.loads(response.text)


In [6]:
df = pd.DataFrame.from_dict(data['data']).set_index(
    'DATE').sort_index(ascending=True)


## Preprocessing

In [7]:
df = df[['CLOSE', 'VOLUME', 'SIGNAL']]


In [8]:
df['ma_20'] = df.CLOSE.rolling(20).mean()


In [9]:
df['open'] = df['high'] = df['low'] = df['CLOSE']


In [10]:
df.reset_index(inplace=True)


In [11]:
df['yearly_high'] = df['CLOSE'].rolling(250).max()
df['yearly_low'] = df['CLOSE'].rolling(250).min()
df['diff'] = df['yearly_high'] - df['yearly_low']
df['distance'] = df['CLOSE'] - df['ma_20']
df['top_5'] = df['distance'] < df['distance'].rolling(
    window=250, min_periods=0).quantile(0.05)


In [12]:
df.columns = ['date', 'close', 'volume', 'signal',
              'open', 'high', 'low', 'yearly_high', 'yearly_low', 'diff', 'ma_20', 'distance', 'top_5']


In [13]:
df.set_index('date', inplace=True)


In [14]:
df = df[['open', 'high', 'low', 'close', 'volume', 'signal', 'ma_20', 'diff', 'distance',
         'yearly_high', 'yearly_low', 'top_5']]


In [15]:
df['top_5'] = df.top_5.astype(dtype=int)


In [16]:
df


,open,high,low,close,volume,signal,ma_20,diff,distance,yearly_high,yearly_low,top_5
date,,,,,,,,,,,,
2018-04-07,NaN,6884.477051,6884.477051,6884.477051,1.840465e+09,0,NaN,NaN,NaN,6884.477051,NaN,0
2018-04-08,NaN,6984.809082,6984.809082,6984.809082,1.856418e+09,0,NaN,NaN,NaN,6984.809082,NaN,0
2018-04-09,NaN,6778.491699,6778.491699,6778.491699,1.413594e+09,0,NaN,NaN,NaN,6778.491699,NaN,0
2018-04-10,NaN,6829.316406,6829.316406,6829.316406,2.437633e+09,0,NaN,NaN,NaN,6829.316406,NaN,0
2018-04-11,NaN,6925.102051,6925.102051,6925.102051,1.550559e+09,0,NaN,NaN,NaN,6925.102051,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-17,28181.6735,27010.000000,27010.000000,27010.000000,9.409536e+09,0,14667.3,15838.49,-1171.6735,27010.000000,30505.79,0
2023-05-18,28077.5810,27395.000000,27395.000000,27395.000000,1.214183e+10,0,14667.3,15838.49,-682.5810,27395.000000,30505.79,0
2023-05-19,27954.9105,26890.000000,26890.000000,26890.000000,1.399245e+10,0,14667.3,15838.49,-1064.9105,26890.000000,30505.79,0


In [17]:
df.to_csv('BTC-USD_extra.csv')


---
## Backtesting

In [18]:
import backtrader.analyzers as btanalyzers
from backtrader.feeds import GenericCSVData
import datetime


In [19]:
class Generic_CSV(GenericCSVData):
    '''
    extends the CSV format beyond the usual OHLC values
    creates lines for other columns in your data.
    '''
    # linesoverride = True # used to completely override OHLC format and define your own
    '''
    'datetime' must always be the last value in lines
    what name your date column has, doesn't matter. you always address it as 'datetime' here.
    keep all the line-names lowercase and reference them later in the code using lowercase
    '''
    lines = ('signal', 'ma_20', 'diff', 'distance',
             'yearly_high', 'yearly_low', 'top_5')
    params = (('signal', 6), ('ma_20', 7), ('diff', 8), ('distance', 9), ('yearly_high',
              10), ('yearly_low', 11), ('top_5', 12))  # by default datetime is column 0


In [20]:
class myStrategy(bt.Strategy):
    '''
    Used to create new customized strategy by extending the base bt.Strategy Class
    '''

    # tuple to store parameter values. access them using self.p.p-name
    params = (
        ('stopwin', 0.3),
        ('stoploss', 0.05),
        ('fast_sma', 20),
        ('mid_sma', 50)
    )

    # function which prints any activity to the console.
    def log(self, txt, dt=None):
        # we have referenced above datetime here as datetime
        dtstr = self.data.datetime.date().isoformat()
        txt = 'Date: {}, {}'.format(dtstr, txt)
        print(txt)

    def __init__(self):
        # store the column 'close' to instantly get the closing price by using self.dataclose[0]
        # could either do this or self.data.col-name[0]
        self.dataclose = self.datas[0].close
        # to keep track of any pending orders, last buy-price and commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        # Add MovingAverageSimple indicators
        self.price_sma_fast = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.fast_sma, plot=False)
        self.price_sma_slow = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=100, plot=False)
        self.price_sma_mid = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.mid_sma, plot=True)

    def notify_order(self, order):
        '''
        invoked after self.buy() or self.sell()
        '''

        if order.status in [order.Submitted, order.Accepted]:
            # If Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            # Check if an order has been completed
            if order.isbuy():
                if self.position.size > 0:  # Checking If we've taken a long position
                    # Take note of buy-price, commission, Max/Min Sell price
                    self.buyprice = order.executed.price
                    self.buycomm = order.executed.comm
                    self.max_sell_price = order.executed.price * \
                        (1+self.params.stopwin)  # 30% stop-profit
                    self.min_sell_price = order.executed.price * \
                        (1-self.params.stoploss)  # 5% stop-loss
                    self.log(
                        'Long Position Taken. Price: {}, Cost: {}, Commission: {}, Max Sell: {}, Min Sell: {}, Position Size: {}'.format(
                            order.executed.price,
                            order.executed.value,
                            order.executed.comm,
                            self.max_sell_price,
                            self.min_sell_price,
                            self.position.size)
                    )

                if self.position.size == 0:  # If cleared a short position
                    self.log('Short Position Cleared. Price: {}, Cost: {}, Commission: {}'.format(
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm
                    ))

            elif order.issell():
                if self.position.size < 0:  # If taken a short position
                    self.min_sell_price = order.executed.price * \
                        (1-self.params.stopwin)  # 30% stop-profit
                    self.max_sell_price = order.executed.price * \
                        (1+self.params.stoploss)  # 5% stop-loss
                    self.log('Short Position Taken. Price: {}, Cost: {}, Commission: {}, Max Sell: {}, Min Sell: {}, Position Size: {}'.format(
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                        self.max_sell_price,
                        self.min_sell_price,
                        self.position.size
                    ))
                if self.position.size == 0:  # If cleared a long position
                    self.log('Long Position Cleared. Price: {}, Cost: {}, Commission: {}, Position Size: {}'.format(
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                        self.position.size))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        '''
        trade refers to a buy/sell pair
        invoked after a buy or sell order is executed i.e. completed.
        '''

        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        '''
        this method is called whenever each new bar (next row in data) is encountered
        '''

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        # checking if we're holding the asset (we've taken a position i.e. self.position.size = 0)
        if not self.position:
            # if (self.dataclose[0] > (self.data.yearly_low + 0.2 * self.data.diff)) and (self.dataclose[0] < (self.data.yearly_low + 0.8 * self.data.diff)):
            #     # buy condition : long
            #     if self.datas[0].top_5[0] == 1 and self.dataclose[0] < self.price_sma_fast[0]:
            if self.dataclose[0] > self.price_sma_mid[0]:
                # if True:#self.dataclose[0] > self.bollinger_bands.lines.top and self.vol_cond:
                self.log('Long Position Entry Signalled. Price: {} Position Size: {}'.format(
                    self.dataclose[0], self.position.size
                ))
                self.order = self.buy()
            # long condition
            # and self.vol_sma_fast[0] > self.vol_sma_slow[0]):
            # if (self.dataclose[-1] < self.bollinger_bands.lines.top[-1] and self.dataclose[0] > self.bollinger_bands.lines.top[0]):
            #     if self.dataclose[0] > self.price_sma_fast[0]:
            #         self.log('Long Position Entry Signalled. Price: {} Position Size: {}'.format(
            #             self.dataclose[0], self.position.size
            #         ))
            #         self.order = self.buy()

            # elif self.dataclose[0] < self.price_sma_fast[0]: # short condition
            #     self.log('Short Position Entry Signalled. Price: {}, Position Size: {}'.format(
            #         self.dataclose[0], self.position.size
            #         ))
            #     self.order = self.sell()

            # (self.dataclose[0] > (self.data.yearly_low + 0.2 * self.data.diff)) and (self.dataclose[0] < (self.data.yearly_low + 0.8 * self.data.diff)):
            elif self.dataclose[0] < self.price_sma_mid[0]:
                # if self.datas[0].top_5[0] == 1 and self.dataclose[0] > self.price_sma_fast[0]:
                self.log('Short Position Entry Signalled. Price: {}, Position Size: {}'.format(
                    self.dataclose[0], self.position.size
                ))
                self.order = self.sell()

        else:  # Clear Position

            if self.position.size > 0:  # If long

                if self.dataclose[0] >= self.max_sell_price:
                    self.log('Long Position Clear Signal. Stopwin Reached. Price: {} Position Size: {}'.format(
                        self.dataclose[0], self.position.size))

                    # Keep track of the created order to avoid a 2nd order
                    self.order = self.sell()

                elif self.dataclose[0] <= self.min_sell_price:
                    self.log('Long Position Clear Signal. Stoploss Reached. Price: {} Position Size: {}'.format(
                        self.dataclose[0], self.position.size))

                    # Keep track of the created order to avoid a 2nd order
                    self.order = self.sell()

            elif self.position.size < 0:  # if short

                if self.dataclose[0] >= self.max_sell_price:
                    self.log('Short Position Clear Signal. Stoploss reached. Price: {}, Position Size: {}'.format(
                        self.dataclose[0], self.position.size
                    ))
                    self.order = self.buy()

                elif self.dataclose[0] <= self.min_sell_price:
                    self.log('Short Position Clear Signal. Stopwin reached. Price: {}, Position Size: {}'.format(
                        self.dataclose[0], self.position.size
                    ))
                    self.order = self.buy()

    def stop(self):
        '''executes after strategy is done executing i.e. at the very end'''
        print('stop executed')
        drawdown = self.analyzers.myDrawdown.get_analysis()
        print('max drawdown: {}'.format(drawdown.max.drawdown))


In [21]:
data = Generic_CSV(
    dataname='BTC-USD_extra.csv',
    fromdate=datetime.datetime(2018, 4, 7),
    todate=datetime.datetime(2023, 5, 21),
    dtformat=('%Y-%m-%d'),
    datetime=0
)


In [22]:
cerebro = bt.Cerebro(stdstats=False, cheat_on_open=True)

cerebro.addobserver(bt.observers.Broker, plot=False)
cerebro.broker.set_coo(True)
cerebro.addobserver(bt.observers.Trades)
cerebro.addobserver(bt.observers.BuySell)

cerebro.adddata(data)

cerebro.addstrategy(myStrategy)

cerebro.broker.setcash(60000)

# 0.1% ... divide by 100 to remove the %
cerebro.broker.setcommission(commission=0.001)

cerebro.addobserver(bt.observers.DrawDown)

cerebro.addobserver(bt.observers.Benchmark,
                    timeframe=bt.TimeFrame.Years)

cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='myTradeAnalyzer')
cerebro.addanalyzer(btanalyzers.SharpeRatio_A,
                    _name='mySharpe', riskfreerate=0.07)

cerebro.addanalyzer(btanalyzers.DrawDown, _name='myDrawdown')


In [23]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

backtest_result = cerebro.run()

print('Sharpe Ratio: {}'.format(
    backtest_result[0].analyzers.mySharpe.get_analysis()['sharperatio']))
print('total trades:', backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['total']['total'])
print('profit/winning trade:', backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['won']['pnl']['average'])
print('loss/losing trade:', backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['lost']['pnl']['average'])
print('PnL:', backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['pnl']['net']['total'])
print('Avg PnL/Trade:', backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['pnl']['net']['average'])


print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())


Starting Portfolio Value: 60000.00
Date: 2018-07-15, Short Position Entry Signalled. Price: 6349.6958007812, Position Size: 0
Date: 2018-07-16, Short Position Taken. Price: 6412.871386718731, Cost: -6412.871386718731, Commission: 6.412871386718731, Max Sell: 6733.514956054668, Min Sell: 4489.009970703111, Position Size: -1
Date: 2018-07-16, Short Position Clear Signal. Stoploss reached. Price: 6741.6918945312, Position Size: -1
Date: 2018-07-17, Short Position Cleared. Price: 6472.18820800779, Cost: -6412.871386718731, Commission: 6.472188208007791
Date: 2018-07-17, OPERATION PROFIT, GROSS -59.32, NET -72.20
Date: 2018-07-17, Long Position Entry Signalled. Price: 7303.373046875 Position Size: 0
Date: 2018-07-18, Long Position Taken. Price: 6544.9273437499805, Cost: 6544.9273437499805, Commission: 6.544927343749981, Max Sell: 8508.405546874976, Min Sell: 6217.680976562481, Position Size: 1
Date: 2018-08-10, Long Position Clear Signal. Stoploss Reached. Price: 6166.5903320312 Position Si

In [24]:
cerebro.plot(iplot=False)


[[<Figure size 1440x811 with 5 Axes>]]

---
- lack of hourly data hence less trades
- in hourly data under same strategy, pnl was negative
---
- Explain the columns of df
- 